In [1]:
!pip install mysql-connector-python pandas


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="992277",
    database="retail_analytics",
    allow_local_infile=True
)

cursor = conn.cursor()
print("Connected to MySQL successfully ✅")


Connected to MySQL successfully ✅


In [2]:
df = pd.read_csv(
    r"D:/Projects/SQL_Retail_Analytics_Project/Dataset/online_retail.csv",
    encoding="latin1"
)

df.head()


,ï»¿InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [3]:
df.shape

(541909, 8)

In [4]:
df.columns

Index(['ï»¿InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [5]:
df.isnull().sum()

ï»¿InvoiceNo         0
StockCode            0
Description       1454
Quantity             0
InvoiceDate          0
UnitPrice            0
CustomerID      135080
Country              0
dtype: int64

In [6]:
df['InvoiceDate'] = pd.to_datetime(
    df['InvoiceDate'],
    format='%d-%m-%Y %H:%M',
    errors='coerce'
)

In [7]:
df['InvoiceDate'].isna().sum()

np.int64(0)

In [8]:
df.dtypes

ï»¿InvoiceNo            object
StockCode               object
Description             object
Quantity                 int64
InvoiceDate     datetime64[ns]
UnitPrice              float64
CustomerID             float64
Country                 object
dtype: object

In [9]:
df['CustomerID'].isna().sum()

np.int64(135080)

In [10]:
len(df)

541909

In [11]:
#Create a cleaned dataframe
df_clean = df.dropna(subset=['CustomerID'])

In [12]:
#Verify size after cleaning 
len(df_clean)


406829

In [13]:
#Double-check no missing CustomerID
df_clean['CustomerID'].isna().sum()

np.int64(0)

In [15]:
df_clean.columns

Index(['ï»¿InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [16]:
#Fix the column name
df_clean = df_clean.rename(columns={'ï»¿InvoiceNo': 'InvoiceNo'})

In [17]:
df_clean.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [18]:
#Check how many cancelled invoices exist
df_clean['InvoiceNo'].astype(str).str.startswith('C').sum()

np.int64(8905)

In [19]:
#Create final clean dataset (no cancellation)
df_final = df_clean[~df_clean['InvoiceNo'].astype(str).str.startswith('C')]

In [20]:
#Verify size after removal
len(df_final)


397924

In [22]:
# Create TotalAmount to represent revenue per transaction
# This is a key business metric used in all revenue and sales analysis
df_final.loc[:, 'TotalAmount'] = df_final['Quantity'] * df_final['UnitPrice']

In [23]:
df_final[['Quantity', 'UnitPrice', 'TotalAmount']].head()

,Quantity,UnitPrice,TotalAmount
0,6,2.55,15.30
1,6,3.39,20.34
2,8,2.75,22.00
3,6,3.39,20.34
4,6,3.39,20.34


In [25]:
# Extract Year from InvoiceDate for time-based trend analysis
# Enables yearly revenue and customer behavior analysis in SQL
df_final.loc[:, 'Year'] = df_final['InvoiceDate'].dt.year

In [26]:
df_final[['InvoiceDate', 'Year']].head()

,InvoiceDate,Year
0,2010-12-01 08:26:00,2010
1,2010-12-01 08:26:00,2010
2,2010-12-01 08:26:00,2010
3,2010-12-01 08:26:00,2010
4,2010-12-01 08:26:00,2010


In [28]:
# Extract Month from InvoiceDate to analyze monthly sales trends
# Useful for seasonality and performance comparison
df_final.loc[:, 'Month'] = df_final['InvoiceDate'].dt.month

In [29]:
df_final[['InvoiceDate', 'Year', 'Month']].head()

,InvoiceDate,Year,Month
0,2010-12-01 08:26:00,2010,12
1,2010-12-01 08:26:00,2010,12
2,2010-12-01 08:26:00,2010,12
3,2010-12-01 08:26:00,2010,12
4,2010-12-01 08:26:00,2010,12


In [30]:
# Final shape of cleaned and feature-engineered dataset
# Confirms readiness for SQL-based analysis
df_final.shape

(397924, 11)

In [31]:
# Export the final cleaned dataset
#This dataset has:
# -No missing CustomerID values
# -No cancelled invoices
# -Proper datetime formatting
# -Engineered features (TotalAmount, Year, Month)
# It is now fully SQL-ready and suitable for analysis and GitHub showcase

df_final.to_csv(
    "D:/Projects/SQL_Retail_Analytics_Project/Dataset/online_retail_clean.csv",
    index=False
)

## Creating Sample Dataset for GitHub (10%)

This step creates a smaller, reproducible version of the cleaned dataset  
so it can be uploaded to GitHub without size limitations.


In [7]:
import pandas as pd

# Load the fully cleaned dataset
df = pd.read_csv(
    r"D:\Projects\SQL_Retail_Analytics_Project\Dataset\online_retail_clean.csv"
)

# Create a 10% reproducible random sample for GitHub upload
df_sample = df.sample(frac=0.10, random_state=42)

# Save the sampled dataset separately (GitHub-friendly size)
df_sample.to_csv(
    r"D:\Projects\SQL_Retail_Analytics_Project\Dataset\sample_online_retail_clean.csv",
    index=False
)

# Check original vs sample dataset shape
df.shape, df_sample.shape



((397924, 11), (39792, 11))